## Oversikt

Formål: Søke i store Parquet-datasett i skyen og produsere et sammenslått byggdatasett fra FKB åpen og OSM som grunnlag for videre analyser.

Forutsetninger: Kjør i Dev Container. Kjøringen kan opprette 4–5 GB lokale filer.


### Om DuckDB i Python og hvorfor vi bruker det her

- **DuckDB Python-biblioteket** lar oss kjøre SQL direkte mot lokale filer og sky-lagrede Parquet (via utvidelser) og få resultatene som DataFrame/Arrow. Vi bruker det som et **mellomlag** for å gjøre det enkelt å materialisere utvalg og vise kart direkte i notebooken.
- Hadde vi ikke trengt interaktiv visuell sjekk, kunne vi kjørt SQL-spørringene «rett gjennom» og hoppet over materialisering/visualisering.

**`con.execute` vs `con.sql` (når bruke hva?)**
- **`con.execute(sql, params?)`**: Utfører spørringen med en gang. Passer for `SET/INSTALL/LOAD/CREATE/COPY/INSERT` og lignende DDL/DML, eller når du vil hente rader eksplisitt med `fetchall()/fetch_df()/fetch_arrow_table()`. Returnerer tilkoblingen.
- **`con.sql(sql)`**: Returnerer en **Relation** (lazy) som kan kjedes og først materialiseres når du kaller `.df()`, `.arrow()`, `.show()`, eller skriver til fil. Foretrekkes når du skal viderebehandle/visualisere resultatet.

Kort tommelfingerregel: DDL/konfig → `execute`; dataflyt/visualisering → `sql(...).df()`.

Lenker: [DuckDB Python API](https://duckdb.org/docs/api/python/overview.html), [Execute SQL / Relation-API](https://duckdb.org/docs/stable/guides/python/execute_sql.html), [DB-API (`execute`/fetch)](https://duckdb.org/docs/stable/clients/python/dbapi.html)


In [14]:
import duckdb
import keplergl

In [15]:
# Ingen behov for .env-fil eller connection string - bruker offentlig tilgang

### Koble til DuckDB og ytelsesinnstillinger

Opprett lokal database (`buildings.db`) og sett antall tråder for å utnytte CPU-en.


In [16]:
con = duckdb.connect("buildings.db")
con.execute("SET threads TO 12;")  # juster denne i forhold til antall kjerner

### Installer DuckDB-utvidelser

Last inn `spatial` (geometri) og `azure` (blobtilgang) for nødvendig funksjonalitet.


In [17]:
con.execute("INSTALL spatial; LOAD spatial");
con.execute("INSTALL azure; LOAD azure;");

### Konfigurer Azure-tilgang

Konfigurer DuckDB for å lese Parquet direkte fra Azure blob storage (`az://`) ved hjelp av offentlig tilgang med kun kontonavn.


In [18]:
# Oppretter Azure secret med kun kontonavn for offentlig tilgang
con.execute("""
CREATE SECRET secret (
    TYPE azure,
    PROVIDER config,
    ACCOUNT_NAME 'doppablobstorage'
);
""")

# Setter curl som transport for Azure-forespørsler
con.execute("SET azure_transport_option_type = curl")

### Forhåndsvisning av data i skyen

Les et lite utvalg fra OSM Parquet på `az://` for å bekrefte tilgang og skjema.


In [19]:
con.sql("""
    SELECT * FROM read_parquet('az://raw/release/2025-11-06.3/dataset=osm/theme=buildings/region=18/**.parquet') LIMIT 10
""")

┌──────────────┬────────────────┬───────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Visuell sanity check

Rask visning av FKB og OSM (utvalgt region) i Kepler.gl for å forstå dataene før matching.


In [7]:

#kart = keplergl.KeplerGl(height=700)
#
#kart.add_data(
#    con.sql("""
#        SELECT * EXCLUDE(geometry), 
#               ST_AsText(geometry) as geometry 
#        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=fkb/theme=buildings/region=46/**.parquet')
#    """).df(), 
#    name='FKB Buildings'
#)
#
#kart.add_data(
#    con.sql("""
#        SELECT * EXCLUDE(geometry), 
#               ST_AsText(geometry) as geometry 
#        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=osm/theme=buildings/region=46/**.parquet')
#    """).df(), 
#    name='OSM Buildings'
#)
#
##kart


### Bygg sammenslått byggdatasett

**Logikk:**
- Behold alle FKB-bygg (autoritative)
- Legg til OSM-bygg uten overlapp med FKB
- Legg til OSM-bygg som avviker betydelig (IoU < 70%)

**Hva er IoU (Intersection over Union)?**
IoU måler hvor like to bygninger er ved å sammenligne overlappende areal med totalt areal. Tenk på det som to sirkler i et Venn-diagram:
- IoU = (overlappende del) / (begge sirkler til sammen)
- 100% = identiske bygninger
- 0% = ingen overlapp

**Hvordan fungerer SQL-spørringen?**
1. **Grid-filtrering:** Deler kartet i rutenett for å raskt finne bygninger som kan overlappe (unngår å sjekke alle kombinasjoner)
2. **Overlapp-beregning:** For hvert OSM-bygg som overlapper med FKB, beregn IoU
3. **Gruppering:** Hvis et OSM-bygg overlapper med flere FKB-bygg, ta den høyeste IoU-verdien (mest lik)
4. **Filtrering:** Behold kun OSM-bygg som er under 70% like (betydelige forskjeller)


In [8]:
# Filter OSM buildings: remove if very similar to FKB, keep if significantly different
# Uses grid-based prefiltering for efficiency (data already in EPSG:4326)
# Uses IoU (Intersection over Union) for robust similarity measurement
con.execute("""
    CREATE OR REPLACE TABLE filtered_buildings AS
    WITH fkb AS (
        SELECT 
            geometry,
            CAST(FLOOR(ST_X(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_x,
            CAST(FLOOR(ST_Y(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_y,
            ST_Area(geometry) as area
        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=fkb/theme=buildings/region=*/**.parquet')
    ),
    osm AS (
        SELECT 
            geometry,
            CAST(FLOOR(ST_X(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_x,
            CAST(FLOOR(ST_Y(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_y,
            ST_Area(geometry) as area
        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=osm/theme=buildings/region=*/**.parquet')
    ),
    -- Find OSM buildings that intersect with FKB buildings (prefiltered by grid cell)
    -- Calculate IoU (Intersection over Union) for robust similarity measure
    -- Group by OSM geometry to get MAX IoU when one OSM building overlaps multiple FKB buildings
    osm_with_overlap AS (
        SELECT 
            o.geometry as osm_geom,
            MAX(
                ST_Area(ST_Intersection(o.geometry, f.geometry)) / 
                ST_Area(ST_Union(o.geometry, f.geometry))
            ) as max_iou_ratio
        FROM osm o
        JOIN fkb f ON o.grid_x = f.grid_x AND o.grid_y = f.grid_y  -- Prefilter using grid
        WHERE ST_Intersects(o.geometry, f.geometry)
        GROUP BY o.geometry  -- Critical: aggregate per OSM building to handle multiple FKB overlaps
    )
    -- Keep all FKB buildings (red)
    SELECT 
        geometry,
        'fkb' as source,
        1 as color,
        NULL as iou_ratio
    FROM fkb
    
    UNION ALL
    
    -- Keep OSM buildings with no overlap (green)
    SELECT 
        o.geometry,
        'osm_no_overlap' as source,
        2 as color,
        0.0 as iou_ratio
    FROM osm o
    WHERE NOT EXISTS (
        SELECT 1 FROM fkb f 
        WHERE f.grid_x = o.grid_x AND f.grid_y = o.grid_y 
        AND ST_Intersects(o.geometry, f.geometry)
    )
    
    UNION ALL
    
    -- Keep OSM buildings with significant difference (IoU < 70%, yellow)
    SELECT 
        osm_geom as geometry,
        'osm_different' as source,
        3 as color,
        max_iou_ratio as iou_ratio
    FROM osm_with_overlap
    WHERE max_iou_ratio < 0.70
""")

# Show summary with IoU statistics
con.sql("""
    SELECT 
        source,
        COUNT(*) as count,
        ROUND(AVG(iou_ratio), 3) as avg_iou,
        ROUND(MIN(iou_ratio), 3) as min_iou,
        ROUND(MAX(iou_ratio), 3) as max_iou,
        CASE source
            WHEN 'fkb' THEN 'FKB buildings (red)'
            WHEN 'osm_no_overlap' THEN 'OSM buildings - no FKB overlap (green)'
            WHEN 'osm_different' THEN 'OSM buildings - significantly different from FKB (yellow)'
        END as description
    FROM filtered_buildings
    GROUP BY source
    ORDER BY count DESC
""").show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────┬─────────┬─────────┬─────────┬─────────┬───────────────────────────────────────────────────────────┐
│     source     │  count  │ avg_iou │ min_iou │ max_iou │                        description                        │
│    varchar     │  int64  │ double  │ double  │ double  │                          varchar                          │
├────────────────┼─────────┼─────────┼─────────┼─────────┼───────────────────────────────────────────────────────────┤
│ osm_no_overlap │ 4154345 │     0.0 │     0.0 │     0.0 │ OSM buildings - no FKB overlap (green)                    │
│ fkb            │   10882 │    NULL │    NULL │    NULL │ FKB buildings (red)                                       │
│ osm_different  │     313 │   0.153 │     0.0 │   0.698 │ OSM buildings - significantly different from FKB (yellow) │
└────────────────┴─────────┴─────────┴─────────┴─────────┴───────────────────────────────────────────────────────────┘



### Drivere (valgfritt)

List tilgjengelige GDAL/Spatial-drivere i DuckDB. Nyttig for feilsøking og å forstå kapabiliteter.


In [9]:
con.sql("""SELECT * FROM ST_Drivers()""").show(max_rows=100)

┌────────────────┬──────────────────────────────────────────────────────┬────────────┬──────────┬──────────┬────────────────────────────────────────────────────┐
│   short_name   │                      long_name                       │ can_create │ can_copy │ can_open │                      help_url                      │
│    varchar     │                       varchar                        │  boolean   │ boolean  │ boolean  │                      varchar                       │
├────────────────┼──────────────────────────────────────────────────────┼────────────┼──────────┼──────────┼────────────────────────────────────────────────────┤
│ ESRI Shapefile │ ESRI Shapefile                                       │ true       │ false    │ true     │ https://gdal.org/drivers/vector/shapefile.html     │
│ MapInfo File   │ MapInfo File                                         │ true       │ false    │ true     │ https://gdal.org/drivers/vector/mitab.html         │
│ UK .NTF        │ UK .NTF  

### Lagring av resultater

Persistér til Parquet for gjenbruk:
- `filtered_buildings` → `buildings.parquet`
- Full OSM → `osm.parquet`
- Full FKB → `fkb.parquet`

Bruker ZSTD-komprimering for god balanse mellom plass og ytelse.


In [10]:
# write filtered_buildings to geoparquet file
con.sql("""
    COPY filtered_buildings
    TO 'buildings.parquet' 
    WITH (FORMAT PARQUET, COMPRESSION 'ZSTD');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
# write osm to geoparquet file
con.sql("""
    COPY (
        SELECT * FROM read_parquet('az://raw/release/2025-11-06.4/dataset=osm/theme=buildings/region=*/**.parquet')
    )
    TO 'osm.parquet' 
    WITH (FORMAT PARQUET, COMPRESSION 'ZSTD');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
# write fkb to geoparquet file
con.sql("""
    COPY (
        SELECT * FROM read_parquet('az://raw/release/2025-11-06.4/dataset=fkb/theme=buildings/region=*/**.parquet')
    )
    TO 'fkb.parquet' 
    WITH (FORMAT PARQUET, COMPRESSION 'ZSTD');
""")

### Visualisering av filtrerte lag

Visualiser filtrerte bygglag med ulike farger i Kepler.gl for rask visuell validering av resultatet.


In [13]:
#
## Visualize filtered buildings on map with separate colored layers
#kart2 = keplergl.KeplerGl(height=700)
#
## Add FKB buildings (red)
#kart2.add_data(
#    con.sql("""
#        SELECT * EXCLUDE(geometry), 
#               ST_AsText(geometry) as geometry 
#        FROM filtered_buildings
#        WHERE source = 'fkb'
#    """).df(), 
#    name='FKB Official (Red)'
#)
#
## Add OSM unique buildings (green)
#kart2.add_data(
#    con.sql("""
#        SELECT * EXCLUDE(geometry), 
#               ST_AsText(geometry) as geometry 
#        FROM filtered_buildings
#        WHERE source = 'osm_no_overlap'
#    """).df(), 
#    name='OSM Unique (Green)'
#)
#
## Add OSM different buildings (orange)
#kart2.add_data(
#    con.sql("""
#        SELECT * EXCLUDE(geometry), 
#               ST_AsText(geometry) as geometry 
#        FROM filtered_buildings
#        WHERE source = 'osm_different'
#    """).df(), 
#    name='OSM Different (Orange)'
#)
#
##kart2